# Creating a function for calling ELVIS

Inputs are:

- bbox (required
- year or year range (required)
- jurisdiction / data collection (default all)
- type (LIDAR or Photogrammetry or both)
- height reference

returns:
- dictionary of file URLs
- sorted by jurisdiction

## Sample data return

format: dictionary of dictionaries, with the datasets contained in a list-of-dictionaries


```
    {
        {"ACT Government" : 
            { "AHD" : [ {
                        'index_poly_name': '6586070',
                        'file_name': 'ACT2015_4ppm-C3-AHD_6586070_55_0002_0002.zip',
                        'file_url': 'https://s3-ap-southeast-2.amazonaws.com/act.elvis/Lidar/z55/ACT2015_4ppm-C3-AHD_6586070_55_0002_0002.zip',
                        'file_size': '196205608',
                        'file_last_modified': '20170616',
                        'bbox': '148.74207269637532,-35.50100460313498,148.76372090731323,-35.48265938553811'
                        },
                        ...                                
                        ],
         "Ellipsoidal" : [ {
                            'index_poly_name': '6606054',
                            'file_name': 'ACT2015_4ppm-C3-ELL_6606054_55_0002_0002.zip',
                            'file_url': 'https://s3-ap-southeast-2.amazonaws.com/act.elvis/Lidar/z55/ACT2015_4ppm-C3-ELL_6606054_55_0002_0002.zip',
                            'file_size': '656868607',
                            'file_last_modified': '20170619',
                            'bbox': '148.76728008939003,-35.644880907252094,148.78895968493913,-35.626530623629115'
                            },
                            ...
                            ]
            }
        },
        {
        "NSW Government" : "No Data"
        }
    }
                                            
```





In [1]:
import requests
import json

from shapely.geometry import box
from shapely.geometry import Polygon
from shapely import wkt
import pyproj

import re


#set elvis URL globally
ELVISURL = "https://elvis2018-ga.fmecloud.com/fmedatastreaming/elvis_indexes/ReturnDownloadables.fmw"

def listcollections(elvisresponse):
    """
    Create a dictionary of 'data collections' in ELVIS, like:
    {
    0 : "NSW Government",
    1 : "ACT Government",
    ...
    7 : "Geoscience Audtralia"
    }
    
    this is used to filter collections later
    """
    i = 0
    collections = {}
    while i < len(elvisresponse):
        collections.update( {i : elvisresponse[i]["source"] } )
        i+=1
    return collections

def pointcloudfilter(pointtype):
    """
    Create a regexp filter for finding specific point cloud types.
    
    """
    if pointtype is None:
        pointfilter = ".*"
        
    elif pointtype == "lidar":
        pointfilter = "-LID|lidar"
        
    elif pointtype == "photo":
        pointfilter = "-PHO"
     
    return pointfilter

def parseresponse(responsecontent, sourcekey, pointtype, heightref, year):
    """
    Parse the ELVIS API response to assemble point cloud datasets into a dictionary organised
    by source, then height reference, then collection type (lidar or photogrammetry)

    
    """
    datadict = {}
    heightrefdict = {}
    datalist = []
        
    #need to decide which point clouds to look for.

    pointfilter = pointcloudfilter(pointtype)
        
    if "downloadables" in responsecontent[sourcekey]:
        if "Point Clouds" in responsecontent[sourcekey]["downloadables"]:
            #if no height ref is set
            if heightref is None:
                # we want to loop over each height ref dictionary
                for height in responsecontent[sourcekey]["downloadables"]["Point Clouds"]:
                    print(height)
                    # and in each height ref dict
                    for dataset in responsecontent[sourcekey]["downloadables"]["Point Clouds"][height]:
                        #loop over the list-of-dicts and construct a filtered list-of-dicts
                        if re.search(pointfilter, dataset["file_url"], re.IGNORECASE):
                            if year is not None and str(year) in dataset["file_url"]:
                                datalist.append(dataset)
                            elif year is None:
                                datalist.append(dataset)

                    if datalist:
                        datadict["height"] = datalist
                        datalist = []
                    else:
                        datadict = "No data"



            else:
                for height in responsecontent[sourcekey]["downloadables"]["Point Clouds"]:
                    if heightref in height:
                        for dataset in responsecontent[sourcekey]["downloadables"]["Point Clouds"][height]:
                            if re.search(pointfilter, dataset["file_url"], re.IGNORECASE):
                                if year is not None and str(year) in dataset["file_url"]:
                                    datalist.append(dataset)
                                elif year is None:
                                    datalist.append(dataset)

                        if datalist is not None:
                            datadict["height"] = datalist
                            datalist = []
                        else:
                            datadict = "No data"
                    
                    
        #if so, add the jurisdiction to the dict
        #datasetdict.update( { "source" : value } )
        #datasetdict.update( { "downloadables" : '' } )
        
        #return no data if there are no point clouds
        else:
            datadict = "No data"
        
    #...or no downloadables
    else:
        datadict = "No data"
    
    
    #should return one dict per data source
    return datadict

    
def callelvis(bbox, pointtype=None, collection=None, year=None, heightref=None):

    datasetdict = {}

    #https://elvis2018-ga.fmecloud.com/fmedatastreaming/elvis_indexes/ReturnDownloadables.fmw
    
    # elvisurl = "https://elvis2018-ga.fmecloud.com/fmedatastreaming/elvis_indexes/ReturnDownloadables.fmw"
    # WCS-style bounding box [xmin, ymin, xmax, ymax] declaration to query the system:
    
    #[148.77, -36.35, 148.78, -35.34]
    
    print(bbox)
    bbox = box(bbox[0], bbox[1], bbox[2], bbox[3],ccw=True)
    
    querybox = wkt.dumps(Polygon(bbox), trim=True)
    querybox = querybox.replace('POLYGON ', 'POLYGON')
    querybox = querybox.replace(', ', ',')
    
    elvisparams = "polygon=" + querybox
    print(elvisparams)
    
    """
    elvisparams = {
        "ymin": bbox[1],
        "ymax": bbox[3],
        "xmin": bbox[0],
        "xmax": bbox[2]
        }
    """
    
    # get a JSON response from ELVIS
    apiresponse = requests.get(ELVISURL, params=elvisparams)
    print(apiresponse.url)
    
    responsecontent = json.loads(apiresponse.content)
    
    #bump off one level, we dont need it
    responsecontent = responsecontent["available_data"]
    
    #create a dictionary of collections / data owners
    collections = listcollections(responsecontent)

    # if collection is default / None, we want all the data
    if collection is None:
        # loop over each of the 8 collection agencies...
        # this sets up the {"source": ... } level
        # ...where 'source' is the jurisdiction name
        
        for key, value in collections.items():
            
            thedatasets = parseresponse(responsecontent, key, pointtype, heightref, year)
            
            datasetdict.update( {value : thedatasets })
            
    # otherwise, we want a limited subset of collections
    else:
        for key, value in collections.items():
            if collection in value:
                thedatasets = parseresponse(responsecontent, key, pointtype, heightref, year)
                # if there are downloadables...
                datasetdict.update({ value : thedatasets })
            
    
    #how shall we return the result? A list of datasets in by-region dicts    
    return datasetdict
    

In [2]:
%%time

# get all the data for all of time in this bbox
result = callelvis([148.77, -36.35, 148.78, -35.34])

[148.77, -36.35, 148.78, -35.34]
polygon=POLYGON((148.78 -36.35,148.78 -35.34,148.77 -35.34,148.77 -36.35,148.78 -36.35))
https://elvis2018-ga.fmecloud.com/fmedatastreaming/elvis_indexes/ReturnDownloadables.fmw?polygon=POLYGON((148.78%20-36.35,148.78%20-35.34,148.77%20-35.34,148.77%20-36.35,148.78%20-36.35))
AHD
AHD
Ellipsoidal
CPU times: user 29 ms, sys: 12.2 ms, total: 41.2 ms
Wall time: 3.12 s


In [81]:
result["NSW"]

{'height': [{'index_poly_name': '6606046',
   'file_name': 'ACT2015_4ppm-C3-ELL_6606046_55_0002_0002.zip',
   'file_url': 'https://s3-ap-southeast-2.amazonaws.com/act.elvis/Lidar/z55/ACT2015_4ppm-C3-ELL_6606046_55_0002_0002.zip',
   'file_size': '474210341',
   'file_last_modified': '20170619',
   'bbox': '148.76887110014735,-35.7169778567827,148.79056911937633,-35.698626943144134'},
  {'index_poly_name': '6606048',
   'file_name': 'ACT2015_4ppm-C3-ELL_6606048_55_0002_0002.zip',
   'file_url': 'https://s3-ap-southeast-2.amazonaws.com/act.elvis/Lidar/z55/ACT2015_4ppm-C3-ELL_6606048_55_0002_0002.zip',
   'file_size': '488758105',
   'file_last_modified': '20170619',
   'bbox': '148.7684728175661,-35.6989537062952,148.79016622474904,-35.68060295028003'},
  {'index_poly_name': '6606050',
   'file_name': 'ACT2015_4ppm-C3-ELL_6606050_55_0002_0002.zip',
   'file_url': 'https://s3-ap-southeast-2.amazonaws.com/act.elvis/Lidar/z55/ACT2015_4ppm-C3-ELL_6606050_55_0002_0002.zip',
   'file_size': '5

In [115]:
%%time
# get all the data from the ACT for all of time in this bbox
result = callelvis([148.77, -36.35, 148.78, -35.34], collection="ACT")

AHD
Ellipsoidal
CPU times: user 22.4 ms, sys: 2.63 ms, total: 25.1 ms
Wall time: 1.71 s


In [116]:
result["ACT Government"].keys()

dict_keys(['AHD', 'Ellipsoidal'])

In [117]:
result

{'ACT Government': {'AHD': [{'index_poly_name': '6586070',
    'file_name': 'ACT2015_4ppm-C3-AHD_6586070_55_0002_0002.zip',
    'file_url': 'https://s3-ap-southeast-2.amazonaws.com/act.elvis/Lidar/z55/ACT2015_4ppm-C3-AHD_6586070_55_0002_0002.zip',
    'file_size': '196205608',
    'file_last_modified': '20170616',
    'bbox': '148.74207269637532,-35.50100460313498,148.76372090731323,-35.48265938553811'},
   {'index_poly_name': '6606046',
    'file_name': 'ACT2015_4ppm-C3-AHD_6606046_55_0002_0002.zip',
    'file_url': 'https://s3-ap-southeast-2.amazonaws.com/act.elvis/Lidar/z55/ACT2015_4ppm-C3-AHD_6606046_55_0002_0002.zip',
    'file_size': '475693565',
    'file_last_modified': '20170616',
    'bbox': '148.76887110014735,-35.7169778567827,148.79056911937633,-35.698626943144134'},
   {'index_poly_name': '6606048',
    'file_name': 'ACT2015_4ppm-C3-AHD_6606048_55_0002_0002.zip',
    'file_url': 'https://s3-ap-southeast-2.amazonaws.com/act.elvis/Lidar/z55/ACT2015_4ppm-C3-AHD_6606048_55_00

In [118]:
%%time
# get all the data from the ACT for 2014 in this bbox
result = callelvis([148.77, -36.35, 148.78, -35.34], collection="ACT", year=2014)

AHD
Ellipsoidal
CPU times: user 20.3 ms, sys: 2.7 ms, total: 23 ms
Wall time: 2.86 s


In [119]:
result

{'ACT Government': 'No data'}

In [120]:
%%time
# get all the data from the ACT for 2015 in this bbox
result = callelvis([148.77, -36.35, 148.78, -35.34], collection="ACT", year=2015)

AHD
Ellipsoidal
CPU times: user 20.5 ms, sys: 2.78 ms, total: 23.3 ms
Wall time: 2.65 s


In [121]:
testlist = []
print('yes') if testlist else print('no')

no


In [122]:
len(result["ACT Government"]["AHD"])

19

In [123]:
len(result["ACT Government"]["Ellipsoidal"])

19

In [124]:
result

{'ACT Government': {'AHD': [{'index_poly_name': '6586070',
    'file_name': 'ACT2015_4ppm-C3-AHD_6586070_55_0002_0002.zip',
    'file_url': 'https://s3-ap-southeast-2.amazonaws.com/act.elvis/Lidar/z55/ACT2015_4ppm-C3-AHD_6586070_55_0002_0002.zip',
    'file_size': '196205608',
    'file_last_modified': '20170616',
    'bbox': '148.74207269637532,-35.50100460313498,148.76372090731323,-35.48265938553811'},
   {'index_poly_name': '6606046',
    'file_name': 'ACT2015_4ppm-C3-AHD_6606046_55_0002_0002.zip',
    'file_url': 'https://s3-ap-southeast-2.amazonaws.com/act.elvis/Lidar/z55/ACT2015_4ppm-C3-AHD_6606046_55_0002_0002.zip',
    'file_size': '475693565',
    'file_last_modified': '20170616',
    'bbox': '148.76887110014735,-35.7169778567827,148.79056911937633,-35.698626943144134'},
   {'index_poly_name': '6606048',
    'file_name': 'ACT2015_4ppm-C3-AHD_6606048_55_0002_0002.zip',
    'file_url': 'https://s3-ap-southeast-2.amazonaws.com/act.elvis/Lidar/z55/ACT2015_4ppm-C3-AHD_6606048_55_00

Ok there's an issue here - the data are being doubled up

In [58]:
%%time
# get all the data from the ACT for 2015 in this bbox, referenced to AHD
result = callelvis([148.77, -36.35, 148.78, -35.34], collection="ACT", year=2015, heightref="AHD")

[148.77, -36.35, 148.78, -35.34]
polygon=POLYGON((148.78 -36.35,148.78 -35.34,148.77 -35.34,148.77 -36.35,148.78 -36.35))
https://elvis2018-ga.fmecloud.com/fmedatastreaming/elvis_indexes/ReturnDownloadables.fmw?polygon=POLYGON((148.78%20-36.35,148.78%20-35.34,148.77%20-35.34,148.77%20-36.35,148.78%20-36.35))
CPU times: user 22.7 ms, sys: 3.38 ms, total: 26.1 ms
Wall time: 3.18 s


In [59]:
result["ACT Government"].keys()

dict_keys(['AHD'])

In [60]:
len(result["ACT Government"]["AHD"])

18

In [61]:
%%time
# get all the data from the ACT for 2015 in this bbox, referenced to Ellipsoid
result = callelvis([148.77, -36.35, 148.78, -35.34], collection="ACT", year=2015, heightref="Ellipsoidal")

[148.77, -36.35, 148.78, -35.34]
polygon=POLYGON((148.78 -36.35,148.78 -35.34,148.77 -35.34,148.77 -36.35,148.78 -36.35))
https://elvis2018-ga.fmecloud.com/fmedatastreaming/elvis_indexes/ReturnDownloadables.fmw?polygon=POLYGON((148.78%20-36.35,148.78%20-35.34,148.77%20-35.34,148.77%20-36.35,148.78%20-36.35))
CPU times: user 19.5 ms, sys: 3.44 ms, total: 22.9 ms
Wall time: 2.26 s


In [62]:
result["ACT Government"].keys()

dict_keys(['Ellipsoidal'])

In [63]:
%%time
# get all the data from the ACT for 2015 in this bbox, referenced to Ellipsoid, using a shortcut
result = callelvis([148.77, -36.35, 148.78, -35.34], collection="ACT", year=2015, heightref="Ell")

[148.77, -36.35, 148.78, -35.34]
polygon=POLYGON((148.78 -36.35,148.78 -35.34,148.77 -35.34,148.77 -36.35,148.78 -36.35))
https://elvis2018-ga.fmecloud.com/fmedatastreaming/elvis_indexes/ReturnDownloadables.fmw?polygon=POLYGON((148.78%20-36.35,148.78%20-35.34,148.77%20-35.34,148.77%20-36.35,148.78%20-36.35))
CPU times: user 23 ms, sys: 3.99 ms, total: 26.9 ms
Wall time: 2.63 s


In [64]:
result["ACT Government"].keys()

dict_keys(['Ellipsoidal'])

In [65]:
len(result["ACT Government"]["Ellipsoidal"])

18

In [66]:
bbox = [148.77, -36.35, 148.78, -35.34]
bboxx = geometry.box(bbox[0],bbox[1],bbox[2],bbox[3])

In [57]:
wkt.dumps(Polygon(bboxx))

'POLYGON ((148.7800000000000011 -36.3500000000000014, 148.7800000000000011 -35.3400000000000034, 148.7700000000000102 -35.3400000000000034, 148.7700000000000102 -36.3500000000000014, 148.7800000000000011 -36.3500000000000014))'

In [77]:
import numpy as np
this = np.arange(100, 50, -5)

In [78]:
this

array([100,  95,  90,  85,  80,  75,  70,  65,  60,  55])

In [8]:
%%time
# get all the data from the ACT for 2015 in this bbox, referenced to Ellipsoid, using a shortcut
result = callelvis([130, -38, 160, -33], collection="NSW")

[130, -38, 160, -33]
polygon=POLYGON((160 -38,160 -33,130 -33,130 -38,160 -38))
https://elvis2018-ga.fmecloud.com/fmedatastreaming/elvis_indexes/ReturnDownloadables.fmw?polygon=POLYGON((160%20-38,160%20-33,130%20-33,130%20-38,160%20-38))
AHD
CPU times: user 955 ms, sys: 298 ms, total: 1.25 s
Wall time: 1min 10s


In [21]:
filesize = 0

for file in result["NSW Government - Spatial Services"]["height"]:
    filesize = filesize + int(file["file_size"])

In [30]:
#assuming file size is in bytes...

filekb = filesize/1000
filemb = filesize/1000/1000

In [28]:
490738661/1000/1000

490.73866100000004

In [31]:
filemb

11552669.448942

In [3]:
# tasmania...
# get all the data for all of time in this bbox
result = callelvis([142.8, -45, 149.4, -39])

[142.8, -45, 149.4, -39]
polygon=POLYGON((149.4 -45,149.4 -39,142.8 -39,142.8 -45,149.4 -45))
https://elvis2018-ga.fmecloud.com/fmedatastreaming/elvis_indexes/ReturnDownloadables.fmw?polygon=POLYGON((149.4%20-45,149.4%20-39,142.8%20-39,142.8%20-45,149.4%20-45))
AHD
Ellipsoidal
Orthometric


In [4]:
result

{'NSW Government - DPIE': 'No data',
 'NSW Government - Spatial Services': 'No data',
 'ACT Government': 'No data',
 'QLD Government': 'No data',
 'TAS Government': {'height': [{'index_poly_name': '4305437',
    'file_name': 'Abbotsham2019-C2-AHD_4305437_GDA2020_55.zip',
    'file_url': 'https://s3-ap-southeast-2.amazonaws.com/tas.elvis/lidar/z55/Abbotsham2019-C2-AHD_4305437_GDA2020_55.zip',
    'file_size': '6401296',
    'file_last_modified': '20191201',
    'bbox': '146.16495277839758,-41.215354352206184,146.17699372746554,-41.206433256561716'},
   {'index_poly_name': '4305438',
    'file_name': 'Abbotsham2019-C2-AHD_4305438_GDA2020_55.zip',
    'file_url': 'https://s3-ap-southeast-2.amazonaws.com/tas.elvis/lidar/z55/Abbotsham2019-C2-AHD_4305438_GDA2020_55.zip',
    'file_size': '6324741',
    'file_last_modified': '20191201',
    'bbox': '146.16506730417956,-41.20634740243097,146.17710656656485,-41.19742626534243'},
   {'index_poly_name': '4305439',
    'file_name': 'Abbotsham2019-

In [9]:
filesize = 0
for file in result["TAS Government"]["height"]:
    #print(file)
    filesize = filesize + int(file["file_size"])

In [10]:
filesize

4169022351729

In [13]:
filekb = filesize/1000
filemb = filesize/1000/1000
filegb = filesize/1000/1000/1000

In [14]:
filegb

4169.022351729